In [2]:
EMBEDDINGS_MODEL_NAME = 'all-MiniLM-L6-v2'

In [5]:
!python3.9 -m pip install langchain
!python -m pip install sentence_transformers

  Using cached langchain-0.0.322-py3-none-any.whl.metadata (15 kB)
  Using cached SQLAlchemy-2.0.22-cp39-cp39-macosx_11_0_arm64.whl.metadata (9.4 kB)
  Using cached aiohttp-3.8.6-cp39-cp39-macosx_11_0_arm64.whl.metadata (7.7 kB)
  Using cached anyio-3.7.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
  Using cached dataclasses_json-0.6.1-py3-none-any.whl.metadata (24 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached langsmith-0.0.51-py3-none-any.whl.metadata (10 kB)
  Using cached pydantic-2.4.2-py3-none-any.whl.metadata (158 kB)
  Using cached tenacity-8.2.3-py3-none-any.whl.metadata (1.0 kB)
  Using cached attrs-23.1.0-py3-none-any.whl (61 kB)
  Using cached multidict-6.0.4-cp39-cp39-macosx_11_0_arm64.whl (29 kB)
  Using cached yarl-1.9.2-cp39-cp39-macosx_11_0_arm64.whl (62 kB)
  Using cached frozenlist-1.4.0-cp39-cp39-macosx_11_0_arm64.whl.metadata (5.2 kB)
  Using cached aiosignal-1.3.1

In [6]:
from langchain.embeddings import HuggingFaceEmbeddings

/Users/Arshika/Visual studio code/vector-db/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [7]:
embeddings = HuggingFaceEmbeddings(model_name=EMBEDDINGS_MODEL_NAME)

/Users/Arshika/Visual studio code/vector-db/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Sample Data

In [8]:
import json
data_json = None
with open('data/sample.json', 'r') as jf:
    data_json = json.load(jf)

In [11]:
!pip install jq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 5.5 MB/s eta 0:00:00a 0:00:01


In [12]:
from vedio_data_json_loader import VideoDataJsonLoader
loader = VideoDataJsonLoader(data_json)

## Qdrant

In [13]:
from langchain.vectorstores import Qdrant

In [14]:
docs = loader.load()

In [15]:
print(len(docs), embeddings)

286 client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
) model_name='all-MiniLM-L6-v2' cache_folder=None model_kwargs={} encode_kwargs={} multi_process=False


In [16]:
!pip install qdrant_client

  Using cached httpx-0.25.0-py3-none-any.whl.metadata (7.6 kB)
  Using cached portalocker-2.8.2-py3-none-any.whl.metadata (8.5 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 915.4 kB/s eta 0:00:00 0:00:01
  Using cached protobuf-4.24.4-cp37-abi3-macosx_10_9_universal2.whl.metadata (540 bytes)
  Using cached httpcore-0.18.0-py3-none-any.whl.metadata (18 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 1.1 MB/s eta 0:00:00a 0:00:01
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.7/176.7 kB 948.2 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 1.1 MB/s eta 0:00:0000:0100:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 884.3 kB/s eta 0:00:0000:0100:01
Using cached httpx-0.25.0-py3-none-any.whl (75 kB)
Using cached portalocker-2.8.2-py3-none-any.whl (17 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 1.3 MB/s eta 0:00:00 0:00:01
Using cached httpcore-0.1

In [18]:
url = "http://localhost:6333"
qdrant = Qdrant.from_documents(
    docs,
    embeddings,
    url=url,
    collection_name="VidCollection",
    force_recreate=True
)

In [19]:
query = "What all things are there in skincare and beauty"
hits = qdrant.similarity_search_with_relevance_scores(query, k=100)

In [20]:
for h in hits:
    print(f"{h[0].metadata['creatorName']}: {h[0].page_content} ({h[0].metadata['srcInfo']['srcVideo']}/{h[0].metadata['srcInfo']['srcCont']}) - ({h[1]})")

Ali Andreea: It has a skincare like good skincare ingredients and it will just give you that beautiful glow without adding any coverage. (63e648093e5488454d0244c3/66) - (0.55286235)
Ali Andreea: But if you guys are not sure, please check out my description box where you will have every single product linked along with the colors that correspond to my skin. (63e648093e5488454d0244c3/50) - (0.52985203)
Ali Andreea: My far number one blush for me, it has to be the Persona Cosmetics. They have my heart. (63e648093e5488454d0244c3/101) - (0.5286913)
Ali Andreea: I strongly recommend you the Makeup Forever HD skin as well. (63e648093e5488454d0244c3/33) - (0.5278668)
Ali Andreea: It's like burnt cheeks like you've been in the sun. Another blush that I love, it's in a similar packaging, is the Huda Beauty. (63e648093e5488454d0244c3/109) - (0.48939782)
Ali Andreea: I remember that I've used this on Candice Swenpole who has normal skin, I would say, and it looked fantastic on her. (63e648093e5488

In [21]:
from qdrant_client.http import models
query = "What all things are there in skincare and beauty"
hits = qdrant.similarity_search_with_relevance_scores(query, k=100, filter=models.Filter(
    must=[
        models.FieldCondition(key="metadata.creatorName", match=models.MatchAny(any=['Beauty Within', 'Allure']))
    ]
))

In [22]:
len(hits)

0

In [23]:
for h in hits:
    print(f"{h[0].metadata['creatorName']}: {h[0].page_content} ({h[0].metadata['srcInfo']['srcVideo']}/{h[0].metadata['srcInfo']['srcCont']}) - ({h[1]})")

In [24]:
from qdrant_client.http import models
query = "What all things are there in skincare and beauty"
hits = qdrant.similarity_search_with_relevance_scores(query, k=100, filter=models.Filter(
    must=[
        models.FieldCondition(key="metadata.creatorName", match=models.MatchAny(any=['Beauty Within', 'Allure'])),
        models.FieldCondition(key="metadata.brandsMentioned", match=models.MatchAny(any=['blends', 'girl']))
    ]
))

In [25]:
len(hits)

0

In [ ]:
for h in hits:
    if 'blends' in h[0].metadata['brandsMentioned'] or 'girl' in h[0].metadata['brandsMentioned']:
        print(f"{h[0].metadata['creatorName']}: {h[0].page_content} ({h[0].metadata['srcInfo']['srcVideo']}/{h[0].metadata['srcInfo']['srcCont']}) - ({h[1]})")
    else:
        raise Exception(f"Got hit out of search {h}")